In [1]:
import sys
import os
sys.path.append(os.getcwd()+'/..')
from model.neighborhood_search import GNNPolicy
from DATASET.neighborhood_search import GraphDataset
from utilities import train_cl as train
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR

In [2]:
#set folder
instance = '8_std'
model = 'neighborhood_search'
uc_model = '3bin'

model_save_path = f"../model_save/{uc_model}_{instance}_{model}/"
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(log_save_path, exist_ok=True)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =50
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
data_path= f'../datasets/{instance}/neighborhood/'

train_data = GraphDataset(data_path+f'/train/{uc_model}/')
valid_data = GraphDataset(data_path+f'/valid/{uc_model}/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict,strict=False)
#     print("")

In [6]:
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
PredictModel.apply(init_weights)
print()

In [7]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS//10, eta_min=ETA_MIN)
best_val_loss = 10
cur_epoch = 0

In [8]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, DEVICE, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.3f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, DEVICE, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.3f}")
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_con.pth')
    if epoch%2==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{epoch}.pth')
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 2.423	Epoch 1 Train loss: 1.756	Epoch 2 Train loss: 1.713	Epoch 3 Train loss: 1.284	Epoch 4 Train loss: 1.113	Epoch 5 Train loss: 1.073	Epoch 6 Train loss: 1.060	Epoch 7 Train loss: 1.080	Epoch 8 Train loss: 1.234	Epoch 9 Train loss: 1.172	Epoch 10 Train loss: 1.127	Epoch 11 Train loss: 1.104	Epoch 12 Train loss: 1.088	Epoch 13 Train loss: 1.071	Epoch 14 Train loss: 1.062	Epoch 15 Train loss: 1.050	Epoch 16 Train loss: 1.052	Epoch 17 Train loss: 1.076	Epoch 18 Train loss: 1.076	Epoch 19 Train loss: 1.080	Epoch 20 Train loss: 1.071	Epoch 21 Train loss: 1.074	Epoch 22 Train loss: 1.083	Epoch 23 Train loss: 1.055	Epoch 24 Train loss: 1.031	Epoch 25 Train loss: 1.027	Epoch 26 Train loss: 1.027	Epoch 27 Train loss: 1.025	Epoch 28 Train loss: 1.093	Epoch 29 Train loss: 1.030	Epoch 30 Train loss: 1.128	Epoch 31 Train loss: 1.111	Epoch 32 Train loss: 1.088	Epoch 33 Train loss: 1.052	Epoch 34 Train loss: 1.034	Epoch 35 Train loss: 1.028	Epoch 36 Train loss: 1.029	Epoch 37 Tr